# Coletando daos IBGE, Armazenando no BigQuery, Preenchendo PPT e Disponibilizando em um chatBot do Telegran

**Este Artigo estará disponivel em tres etapas:**
- Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
- Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT
- Terceira Etapa: Disponibilização dos dados no ChatBoot do Telegran

### Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT
Neste Artigo vamos consultar os dados armazenados no BigQuery e preencher o PPT. 

**Importa Libs**

In [165]:
import sidrapy
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
from pptx import Presentation
from pptx.chart.data import CategoryChartData
import pandas as pd


#### Abrindo PPT

In [166]:
pres = Presentation("arqs/pessoasMunicipio.pptx")# pessoasMunicipio.pptx")

#### Seleciona Slide a ser alterado

In [167]:
slides = [slide for slide in pres.slides] 
slide2 = slides[1]

#### Troca o Titulo do Slide

In [168]:
uf = 'SP' # Variavel para armazenar o Estado que ser[a utilizado]
titulo_grafico = [objeto for objeto in slide2.shapes if objeto.has_text_frame]
titulo_grafico[0].text = str(titulo_grafico[0].text).format(uf = uf)

### Autenticando no Google Cloud
**Neste ponto iremos utilizar a chave de serviço do GCP**

**Obs:** Caso não saiba gerar a chave pode utilizar o link a seguir:
- https://medium.com/@alexandretavares.bigdata/criando-uma-conta-de-servi%C3%A7o-no-gcp-eeac59f3cbcb

In [169]:
# Informe aqui o caminho onde a chave estará armazenada
# Recomendamos que guarde esta chave em um cofre digital para maior segurança
pathCredencial = "key/artmedium.json" 

credentials = service_account.Credentials.from_service_account_file(
    pathCredencial,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

#### Busca dados do Municipio informado no BigQuery

In [170]:
QUERY = ( f'''
              DECLARE anos STRING;
              SET anos = (
                SELECT 
                  CONCAT('("', STRING_AGG(DISTINCT ano, '", "'), '")'),
                FROM `artmedium.raw.tb_nat_populacao_municipio`
              );
              EXECUTE IMMEDIATE format("""
                SELECT * FROM
                (
                  SELECT 
                    municipio,
                    ano,
                    valor
                  FROM `artmedium.raw.tb_nat_populacao_municipio`
                  where uf = "{uf}"
                )
                PIVOT
                (
                  sum(valor) as ano
                  FOR ano in %s
                )
                ORDER BY municipio ASC
              """, anos)
          '''
    )
df = pandas_gbq.read_gbq(QUERY, credentials=credentials)
df


Downloading: 100%|██████████| 645/645 [00:00<00:00, 1803.74rows/s]


,municipio,ano_2016,ano_2017,ano_2018,ano_2019,ano_2020,ano_2021
0,Adamantina,35094,35139,35023,35068,35111,35153
1,Adolfo,3616,3609,3571,3562,3554,3545
2,Aguaí,35189,35508,35954,36305,36648,36981
3,Agudos,36704,36880,37023,37214,37401,37582
4,Alambari,5658,5754,5918,6025,6129,6231
...,...,...,...,...,...,...,...
640,Águas de São Pedro,3205,3268,3380,3451,3521,3588
641,Álvares Florence,3824,3795,3712,3679,3647,3616
642,Álvares Machado,24733,24813,24830,24915,24998,25078
643,Álvaro de Carvalho,5075,5119,5179,5227,5274,5320


#### Preenche grafico do Slide 2 com dados do BigQuery

In [171]:
slide2_charts = [shape for shape in slide2.shapes if shape.has_chart]
bar_chart = [shape.chart for shape in slide2_charts ]
chart_data  = CategoryChartData()

colunas = [x for x in df.columns]
colunas.pop(0)

chart_data.categories = [str(x) for x in df['municipio']]
for x in colunas:
    chart_data.add_series(x.split('_')[1], [qtdPessoas for qtdPessoas in df[x]])
bar_chart[0].replace_data(chart_data)
bar_chart[0].chart_title.text_frame.text = "População por Municipio"




#### Preenche Titulo do Slide 3

In [172]:
slide3 = slides[2]
titulo_grafico = [objeto for objeto in slide3.shapes if objeto.has_text_frame]
titulo_grafico[0].text = str(titulo_grafico[0].text).format(uf = uf)

#### Consulta dados do Estado no BigQuery

In [173]:
QUERY2 = ( f'''
DECLARE anos STRING;
SET anos = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT ano, '", "'), '")'),
  FROM `raw.tb_nat_populacao_uf`
);
EXECUTE IMMEDIATE format("""
SELECT * FROM
(
  SELECT 
    uf,
    ano,
    valor
  FROM `raw.tb_nat_populacao_uf`
  where uf = "{uf}"
)
PIVOT
(
  sum(valor) AS pessoas
  FOR ano in %s
)
ORDER BY uf ASC
""", anos)
          '''
    )
df = pandas_gbq.read_gbq(QUERY2, credentials=credentials)
df


Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.25rows/s]


,uf,pessoas_2016,pessoas_2017,pessoas_2018,pessoas_2019,pessoas_2020,pessoas_2021
0,SP,44749699,45094866,45538936,45919049,46289333,46649132


#### Preenche Grafico do Slide 2 com dados do Estado Selecionado

In [174]:
slide3_charts = [shape for shape in slide3.shapes if shape.has_chart]
bar_chart2 = [shape.chart for shape in slide3_charts ]
chart_data2  = CategoryChartData()

colunas = [x for x in df.columns]
colunas.pop(0)
chart_data2.categories = [str(x) for x in df['uf']]
for x in colunas:
    chart_data2.add_series(x.split('_')[1], [qtdPessoas for qtdPessoas in df[x]])
bar_chart2[0].replace_data(chart_data2)
bar_chart2[0].chart_title.text_frame.text = "População por Estado"


#### Salva PPT

In [176]:

pres.save('arqs/pessoasMinicipios_Preenchido.pptx')